In [1]:
from google.colab import drive
drive.mount('/content/drive')
import datetime
import json
import numpy as np
import os
import pandas as pd
import sys 
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from keras.callbacks import History 
history = History()

Mounted at /content/drive


In [2]:
BATCH_SIZE = 10
#number_of_samples = 10000
number_of_balanced_dfs = 1
MIN_number_of_attributes = 15
MAX_number_of_attributes = 15
number_of_NN_repetitions = 3
#number_of_attributes = 20

In [3]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(144, activation='relu'), 
    tf.keras.layers.Dense(144, activation='relu'), 
    tf.keras.layers.Dense(12)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [4]:
for i in range(1,number_of_balanced_dfs + 1):
  #Data processing
  title = f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/shuffled_df_train_{i}_8000_samples.csv'
  df_train_raw = pd.read_csv(title)
  print(title)
  title = f'/content/drive/MyDrive/network-traffic-classification-main/csv-files/shuffled_df_test_{i}_2000_samples.csv'
  df_test_raw = pd.read_csv(title)

  target_train = df_train_raw.pop('target')
  target_test = df_test_raw.pop('target')

# attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/JSON-ATTRIBUTES-FILE-HERE"
# attr_title = "/content/drive/MyDrive/network-traffic-classification-main/nn-attribute-arrays/rankings_last_run"
# with open(attr_title, "r") as f:
#   rep = json.load(f)
#   print(rep['sorted_attributes_by_weights']) 

# attributes_array = rep['sorted_attributes_by_weights']  
# Attributes array sorted ascending by number of unique values
# ['24', '29', '63', '61', '62', '10', 'target', '17', '64', '92', '91', '79', '80', '38', '37', '59', '60', '46', '36', '15', '22', '82', '81', '83', '84', '45', '32', '34', '31', '33', '87', '88', '90', '89', '47', '48']
# attr array by permutation:
#attributes_array = ['48', '47', '89', '90', '88', '87', '33', '31', '34', '32', '45', '84', '83', '81', '82', '22', '15', '36', '46', '60', '59', '37', '38', '80', '79', '91', '92', '64', '17', '10', '62', '61', '63', '29', '24']
  attributes_array = ['82', '83', '81', '89', '15', '22', '84', '90', '88', '79', '87', '80', '17', '64', '60', '10', '29', '59', '63', '24', '62', '61', '91', '48', '47', '32', '34', '31', '33', '45', '46', '36', '37', '38', '92'] 
  

  for number_of_attributes in range(MIN_number_of_attributes ,MAX_number_of_attributes+1):
    # Select a sub-dataframe from the main dataframe with a lower number of attributes
    df_train = df_train_raw[attributes_array[0:number_of_attributes]]
    df_test = df_test_raw[attributes_array[0:number_of_attributes]]

    df_train = np.asarray(df_train).astype(np.float32)
    target_train = np.asarray(target_train).astype(np.float32)
    df_test_np = np.asarray(df_test).astype(np.float32)
    target_test_np = np.asarray(target_test).astype(np.float32)

    # Converts pandas dataframe to tensorflow object
    df_train = tf.convert_to_tensor(df_train)
    # Normalize the data
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(df_train)

    # Execute Trainning and Testing
    print('Starting training for',number_of_attributes,'attributes')
    test_acc = []
    train_acc = []
    for k in range(number_of_NN_repetitions):
      model = get_basic_model()
      history = model.fit(df_train, target_train, epochs=5, verbose=1, batch_size=BATCH_SIZE, validation_split=0.2)
      one_train_acc = history.history['accuracy']
      #print("one_train_acc",one_train_acc)
      #print("one_train_acc",one_train_acc[-1])

      test_loss, one_test_acc =  model.evaluate(df_test_np,  target_test_np, verbose=1, batch_size=BATCH_SIZE)      
      test_acc.append(one_test_acc)
      train_acc.append(one_train_acc[-1])
      tf_predictions_probabilities = model.predict(df_test_np)
      tf_predictions = []

      for j,x in enumerate(tf_predictions_probabilities):
        #print("i:",i,"x_max:",x.max(),"x:",x)  
        j_max = x.argmax()
        tf_predictions.append(j_max)  

      conf_m = tf.math.confusion_matrix(target_test_np,tf_predictions)
      print(conf_m)

    
      
    train_acc_mean = sum(train_acc) / len(train_acc)
    test_acc_mean = sum(test_acc) / len(test_acc)
    print(f'test_acc_mean: {test_acc_mean}, train_acc_mean: {train_acc_mean}')

    # Date for report file
    current_date_and_time = datetime.date.today()
    current_date_and_time_string = str(current_date_and_time)
    # Report file content
    report = {
        "datetime": current_date_and_time_string,
        "attributes": attributes_array[0:df_train.shape[1]],
        "number_of_attributes": number_of_attributes,       
        "accuracy_test": test_acc_mean,
        "accuracy_train": train_acc_mean
        
    }
    # Writes into the file
    title = "/content/drive/MyDrive/network-traffic-classification-main/nn-reports/shuffled_df%d"%(i) + "_%s"%current_date_and_time_string + "_with_%d_attributes.json"%number_of_attributes
    with open(title, "w") as f:
      json.dump(report, f)







/content/drive/MyDrive/network-traffic-classification-main/csv-files/shuffled_df_train_1_8000_samples.csv
Starting training for 15 attributes
Epoch 1/5
7680/7680 [==============================] - 21s 3ms/step - loss: 0.3592 - accuracy: 0.8829 - val_loss: 0.2065 - val_accuracy: 0.9291
Epoch 2/5
7680/7680 [==============================] - 20s 3ms/step - loss: 0.1803 - accuracy: 0.9336 - val_loss: 0.1524 - val_accuracy: 0.9409
Epoch 3/5
7680/7680 [==============================] - 21s 3ms/step - loss: 0.1461 - accuracy: 0.9442 - val_loss: 0.1249 - val_accuracy: 0.9538
Epoch 4/5
7680/7680 [==============================] - 21s 3ms/step - loss: 0.1270 - accuracy: 0.9493 - val_loss: 0.1348 - val_accuracy: 0.9477
Epoch 5/5
2400/2400 [==============================] - 4s 2ms/step - loss: 1.2689 - accuracy: 0.8574
tf.Tensor(
[[1835    3    0    5   84   67    2    2    1    0    1    0]
 [   0 1934   23    7   10    6    6    4    6    0    4    0]
 [   0    3 1972   11    4    1    0    1   